In [1]:
import torchvision.datasets as dsets
import torchvision.transforms as transforms

In [2]:
# Transformations, same as the ViT training scripts if without autoaug policy
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

In [3]:
# Load train set
train_set = dsets.CIFAR10('../data', train=True, download=True, transform=transform_train)

# Load test set (using as validation)
val_set = dsets.CIFAR10('../data', train=False, download=True, transform=transform_test)

Files already downloaded and verified
Files already downloaded and verified


# train vit model on cifar-10

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import os.path
from torch.utils.data import DataLoader
from tqdm.auto import tqdm

In [5]:
from copy import deepcopy

In [6]:
# Select device
device = torch.device('cpu')

### training of the original model (ViT but lighter) is handled by the scripts in directory ViT-CIFAR

we just need to load the model here

In [7]:
from vit import ViT

In [8]:
model_path = '../trained_models/cifar_c10_vit_surrogate.pt'

In [9]:
if os.path.isfile(model_path):
    # load saved model
    print("Loading saved model")
    model = torch.load(model_path).to(device)
else:
    # Create model from trained
    print("Creating model, from elsewhere pretrained")
    model = ViT(in_c = 3, num_classes=10, img_size=32, patch=8, dropout=0.0, num_layers=7,
           hidden=384, mlp_hidden=384, head=12, is_cls_token=True)
    loaded = torch.load("../ViT-CIFAR/weights/vit_c10_aa_ls_note_oriforFastSHAP.pth")
    model_states = {key[6:]:loaded[key] for key in loaded}
    model.load_state_dict(model_states)
    model = model.to(device)

Loading saved model


SAME as the resnet model, ViT model outputs unnormalized logits (scores) for each class, which serve as inputs to the Cross Entropy Loss

In [10]:
torch.save(model.cpu(), model_path)

# train surrogates

In [11]:
from fastshap import ImageSurrogate
from fastshap.utils import MaskLayer2d, DatasetInputOnly, KLDivLoss

In [12]:
surr_model_path = '../trained_models/cifar_c10_vit_surrogate_lb20.pt'

In [13]:
# Check for model
if os.path.isfile(surr_model_path):
    print('Loading saved surrogate model')
    surr = torch.load(surr_model_path).to(device)
    surrogate = ImageSurrogate(surr, width=32, height=32, superpixel_size=2)
else:
    print("Creating new surrogate model")
    surr = nn.Sequential(
        MaskLayer2d(value=0, append=True),
        ViT(in_c = 4, num_classes=10, img_size=32, patch=8, dropout=0.0, num_layers=7,
        hidden=384, mlp_hidden=384, head=12, is_cls_token=True)).to(device)
    # Set up surrogate object
    surrogate = ImageSurrogate(surr, width=32, height=32, superpixel_size=2)

Loading saved surrogate model


In [14]:
# to retrain just go from here 
# print("Creating new surrogate model")
# surr = nn.Sequential(
#     MaskLayer2d(value=0, append=True),
#     ViT(in_c = 4, num_classes=10, img_size=32, patch=8, dropout=0.0, num_layers=7,
#     hidden=384, mlp_hidden=384, head=12, is_cls_token=True)).to(device)
# # Set up surrogate object
# surrogate = ImageSurrogate(surr, width=32, height=32, superpixel_size=2)

In [17]:
# Set up datasets
train_surr = DatasetInputOnly(train_set)
val_surr = DatasetInputOnly(val_set)
original_model = nn.Sequential(model, nn.Softmax(dim=1)).to(device)

In [16]:
# Train
surrogate.train_original_model(
    train_surr,
    val_surr,
    original_model,
    batch_size=256,
    max_epochs=100,
    loss_fn=KLDivLoss(),
    lookback=20, #10
    bar=True,
    verbose=True)

ValueError: too many values to unpack (expected 2)

In [ ]:
surr.cpu()
# torch.save(surr, 'cifar_c10_vit_surrogate_lb20.pt') # lookback 20, best val loss 0.7623
torch.save(surr, surr_model_path) # lookback 20, best val loss, best vak kiss 0.5698
# ls 0.827

# train fastshap

In [18]:
from unet import UNet
from fastshap import FastSHAP

In [ ]:
explainer_path = 'cifar_explainer_unet_lb20.pt'

In [ ]:
# Check for model
if os.path.isfile(explainer_path):
    print('Loading saved explainer model')
    explainer = torch.load(explainer_path).to(device)
    fastshap = FastSHAP(explainer, surrogate, link=nn.LogSoftmax(dim=1))

else:
    # Set up explainer model
    explainer = UNet(n_classes=10, num_down=2, num_up=1, num_convs=3).to(device)

    # Set up FastSHAP object
    fastshap = FastSHAP(explainer, surrogate, link=nn.LogSoftmax(dim=1))

    # Set up datasets
    fastshap_train = DatasetInputOnly(train_set)
    fastshap_val = DatasetInputOnly(val_set)

    # Train
    fastshap.train(
        fastshap_train,
        fastshap_val,
        batch_size=128,
        num_samples=2,
        max_epochs=200,
        eff_lambda=1e-2,
        validation_samples=1,
        lookback=10,
        bar=True,
        verbose=True)
    
    # Save explainer
    explainer.cpu()
    torch.save(explainer, explainer_path)
    explainer.to(device)

# visualization

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# Select one image from each class
dset = val_set
targets = np.array(dset.targets)
num_classes = targets.max() + 1
inds_lists = [np.where(targets == cat)[0] for cat in range(num_classes)]
inds = [np.random.choice(cat_inds) for cat_inds in inds_lists]
x, y = zip(*[dset[ind] for ind in inds])
x = torch.stack(x)

# Get explanations
values = fastshap.shap_values(x.to(device))

# Get predictions
pred = surrogate(
    x.to(device),
    torch.ones(num_classes, surrogate.num_players, device=device)
).softmax(dim=1).cpu().data.numpy()

fig, axarr = plt.subplots(num_classes, num_classes + 1, figsize=(22, 20))

for row in range(num_classes):
    # Image
    classes = ['Airplane', 'Car', 'Bird', 'Cat', 'Deer', 'Dog', 'Frog', 'Horse', 'Ship', 'Truck']
    mean = np.array([0.4914, 0.4822, 0.4465])[:, np.newaxis, np.newaxis]
    std = np.array([0.2023, 0.1994, 0.2010])[:, np.newaxis, np.newaxis]
    im = x[row].numpy() * std + mean
    im = im.transpose(1, 2, 0).astype(float)
    im = np.clip(im, a_min=0, a_max=1)
    axarr[row, 0].imshow(im, vmin=0, vmax=1)
    axarr[row, 0].set_xticks([])
    axarr[row, 0].set_yticks([])
    axarr[row, 0].set_ylabel('{}'.format(classes[y[row]]), fontsize=14)
    
    # Explanations
    m = np.abs(values[row]).max()
    for col in range(num_classes):
        axarr[row, col + 1].imshow(values[row, col], cmap='seismic', vmin=-m, vmax=m)
        axarr[row, col + 1].set_xticks([])
        axarr[row, col + 1].set_yticks([])
        if col == y[row]:
            axarr[row, col + 1].set_xlabel('{:.2f}'.format(pred[row, col]), fontsize=12, fontweight='bold')
        else:
            axarr[row, col + 1].set_xlabel('{:.2f}'.format(pred[row, col]), fontsize=12)
        
        # Class labels
        if row == 0:
            axarr[row, col + 1].set_title('{}'.format(classes[y[col]]), fontsize=14)

plt.tight_layout()
plt.show()

# Validating Surrogate

In [19]:
import surrogate as sg
# from surrogate import validate_all_ones

# Select one image from each class
dset = val_set
targets = np.array(dset.targets)
num_classes = targets.max() + 1
# inds_lists = [np.where(targets == cat)[0] for cat in range(num_classes)]
# inds = [np.random.choice(cat_inds) for cat_inds in inds_lists]
# x, y = zip(*[dset[ind] for ind in inds])
# x = torch.stack(x)

# Get explanations
# values = fastshap.shap_values(x.to(device))

# Get predictions


sg.validate_all_ones(surrogate, loss_fn=KLDivLoss(), val_data=val_surr, num_classes=num_classes, validation_batch_size=256)
    
    
# pred = surrogate(
#     x.to(device),
#     torch.ones(num_classes, surrogate.num_players, device=device)
# ).softmax(dim=1).cpu().data.numpy()

torch.Size([10000, 256])


ValueError: not enough values to unpack (expected 3, got 2)